# Classification

---

Actually, classification is a regression but in narrow (discrete) interval. In classification, we try to classify objects, events, or even people into some category based on their similarities or some common criteria. In regression task we try to find best fitting line to our data, whereas in classification task we also try to find a line which gives us the best separation or discrimination between categories or groups. Compared to regression, in classification the target is not a continuous variable, rather then discrete, taking finite number of values. Such as survived or not survived, defaulted or not defaulted, spam or not spam, and so on. There are three types of classification in a wild:


* **Binary Classification**


* **Multi-Class Classification**


* **Multi-Label Classification**


In this class, we only focus on binary classification. However, you will have solid foundation to dig deeper down in the rabbit whole.


$$
$$


![alt text](images/classification.png "Title")


$$
$$



### Lecture outline

---

* Problem Statement


* Data Description


* EDA - Exploratory Data Analysis


* Data Processing


* Logistic Regression


* Decision Tree Classification


* Random Forest Classification


* Model Performance Assessment

In [ ]:
# For data processing
import pandas as pd
import numpy as np

# For data viz
import matplotlib.pyplot as plt
import seaborn as sns

# For modeling
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# For model performance assessment
from sklearn import metrics

[best kaggle notebook - titanic](https://www.kaggle.com/ash316/eda-to-prediction-dietanic)


[ესეც_ნახე_აუცილებლად](https://www.kaggle.com/mnassrib/titanic-logistic-regression-with-python)


[Titanic Random Forest: 82.78%](https://www.kaggle.com/zlatankr/titanic-random-forest-82-78)

# Resources


https://www.kaggle.com/dmilla/introduction-to-decision-trees-titanic-dataset


https://www.kaggle.com/masumrumi/decision-tree-with-titanic-dataset

#### Reference

